DQN Baseline

Logan Wong

law3082

In [ ]:
# REMINDER: make sure you set
# Runtime
# Change runtime type
# T4 GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
!ls -la

/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari
total 70
drwx------ 2 root root  4096 Sep 25 15:17 checkpoints
drwx------ 2 root root  4096 Sep 25 15:17 code
drwx------ 2 root root  4096 Sep 25 15:04 .git
-rw------- 1 root root 49422 Sep 26 19:27 github_terminal.ipynb
-rw------- 1 root root    33 Sep 26 19:25 .gitignore
drwx------ 2 root root  4096 Sep 25 15:17 models
-rw------- 1 root root    26 Sep 25 15:04 README.md
drwx------ 2 root root  4096 Sep 25 15:17 results


In [3]:
!pip install stable-baselines3 gymnasium[atari,accept-rom-license] ale-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.9 MB/s eta 0:00:00


In [4]:
import os
import torch
import gymnasium as gym
import stable_baselines3
import ale_py
import numpy as np
from stable_baselines3 import DQN


# Visualization
from PIL import Image
import io
import base64
from IPython.display import display, HTML



# For debugging
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback
import time

# Action masking
from gymnasium import ActionWrapper
from stable_baselines3.common.atari_wrappers import AtariWrapper

# Vector environment
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecEnvWrapper, DummyVecEnv


import gc

print("All imports working")

All imports working


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

GPU available: True
GPU: Tesla T4


In [6]:
def convert(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)

In [7]:
# # Show the first frame as an image
# def show_frame(obs):
#     # Convert to PIL Image and display
#     img = Image.fromarray(obs)
#     display(img)

Create Environment and Model

In [8]:
# Action space: Discrete(6)
# Number of actions: 6
# Action meanings: ['NOOP', 'FIRE', 'UP', 'DOWN', 'UPFIRE', 'DOWNFIRE']
# Observation shape: (210, 160, 3)
# Observation space: Box(0, 255, (210, 160, 3), uint8)

# ACTIONS:
# 0: NOOP
# 1: FIRE
# 2: UP
# 3: DOWN

# 4: UPFIRE
# 5: DOWNFIRE

In [9]:
class ActionReducer(ActionWrapper):
  def __init__(self, env):
    super().__init__(env)

    # NOOP, FIRE, UP, and DOWN only. No UPFIRE. No DOWNFIRE.
    self.allowed_actions = [0,1,2,3]

    self.action_space = gym.spaces.Discrete(len(self.allowed_actions))

  def action(self, action):
    return self.allowed_actions[action]

In [10]:
def make_env():
  # DQN only supports single environments (not vectorized)
  env = gym.make("ALE/Bowling-v5")
  env = ActionReducer(env)
  env = Monitor(env)
  # disable reward clipping
  env = AtariWrapper(env, clip_reward=False)
  return env

In [11]:
seed = 316
torch.manual_seed(seed)

env = DummyVecEnv([make_env])
env = VecFrameStack(env, n_stack=4)

model = DQN(
    "CnnPolicy",
    env,
    learning_rate=0.0001,
    # learning_rate=0.00005,
    # buffer_size=100000,
    buffer_size=50000,
    batch_size=32,
    gamma=0.99,
    target_update_interval=1000,
    train_freq=4,
    gradient_steps=1,
    # exploration_final_eps=0.01,
    exploration_final_eps=0.02,
    exploration_fraction=0.1,
    # learning_starts=100000,
    learning_starts=10000,
    verbose=1,
    device="cuda"
)

print("DQN model created")

Using cuda device
Wrapping the env in a VecTransposeImage.
DQN model created


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# # Confirm number of possible actions is smaller now
# print(f"Original action space: {gym.make('ALE/Bowling-v5').action_space}")
# print(f"Reduced action space: {env.action_space}")

Train the model

In [12]:
class SimpleCheckpointCallback(BaseCallback):
    def __init__(self, save_freq, save_path, name_prefix):
        super().__init__()
        self.save_freq = save_freq
        self.save_path = save_path
        self.name_prefix = name_prefix

    def _on_step(self):
        if self.n_calls % self.save_freq == 0:
            path = os.path.join(self.save_path, f"{self.name_prefix}_{self.n_calls}")
            self.model.save(path)
            print(f"Checkpoint saved at step {self.n_calls}")
        return True

In [ ]:
total_timesteps = 10000000    # 10M
# total_timesteps =  2000000    # 2M
# total_timesteps =  1000000    # 1M
# total_timesteps =   100000    # 100K
# total_timesteps =    10000    # 10K
# total_timesteps =     5000    # 5K

# Save couple of steps
checkpoint_callback = CheckpointCallback(
    save_freq=10000,
    save_path="/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari/checkpoints",
    name_prefix=f"dqn_baseline_{total_timesteps}"
)

# Time how long it takes
print("Training started")
start_time = time.time()

# log_interval: Print train metrics every 10 episodes
model.learn(
    total_timesteps=total_timesteps,
    callback=checkpoint_callback,
    progress_bar=True,
    log_interval=10
)
end_time = time.time()
print("Training done")

env.close()

# Calculate run time
training_duration = end_time - start_time
time_in_minutes_and_seconds = convert(training_duration)
print(f"Time taken: {time_in_minutes_and_seconds}")
# print(f"Speed: {total_timesteps/training_duration:.2f} steps/second")

# Save model to Google Drive
trained_model_save_path = f"/content/drive/MyDrive/MECE689_Bowling/MECE689_RL_Bowling_Atari/models/dqn_baseline_{total_timesteps}"
model.save(trained_model_save_path)
print("Model saved to Google Drive")

Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

Training started
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.29e+03 |
|    ep_rew_mean      | 31       |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 10       |
|    fps              | 346      |
|    time_elapsed     | 16       |
|    total_timesteps  | 5655     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.3e+03  |
|    ep_rew_mean      | 26.6     |
|    exploration_rate | 0.989    |
| time/               |          |
|    episodes         | 20       |
|    fps              | 293      |
|    time_elapsed     | 38       |
|    total_timesteps  | 11360    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.24e-05 |
|    n_updates        | 339      |
----------------------------------
----------------------------------
| rollout/            |          |
|  

In [ ]:
# torch.cuda.empty_cache()
# del model
# del env